# technical stuff

In [ ]:
import pandas as pd

In [ ]:
from scipy.stats import wasserstein_distance
import numpy as np
from itertools import tee
from scipy.sparse.csgraph import dijkstra
import histlite as hl

In [ ]:
from numpy import linalg as LA
from scipy.sparse.csgraph import laplacian
import matplotlib.pyplot as plt
import scipy.stats as st

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pwd

%cd ./drive/MyDrive/Colab_Notebooks/

/content
/content/drive/MyDrive/Colab_Notebooks




# Data preparation 




In [ ]:
def data_from_csv(csv):
  data = pd.read_csv(csv)
  data = data.drop(['Unnamed: 0'], axis=1)
  datacorr = data.corr()
  datacorr = 1/datacorr
  abs_datacorr = abs(datacorr)

  abs_datacorr = abs_datacorr.to_numpy()
  np.fill_diagonal(abs_datacorr, 0)
  return abs_datacorr

# Ricci **Flow**

In [ ]:
def pairwise(iterable):
    a, b = tee(iterable)
    next(b, None)
    return zip(a, b)

def surgery(x):
  return True if (x % 5 == 0) else False

In [ ]:
def Ricci_Flow(data, iters = 50):
  pairs = pairwise(range(data.shape[0]))
  old_data = data
    
  for i in range(0, iters):
    dist_matrix = dijkstra(csgraph=old_data, directed=False)
    new_data = np.empty(data.shape)
    max_weight = old_data.max()
    for x, y in pairs:
      emd = wasserstein_distance(old_data[:, x], old_data[:, y])
      shortest_path = dist_matrix[x][y]
      k_xy = 1 - emd/shortest_path
      new_weight = shortest_path - k_xy * shortest_path

      if surgery(i):
        if new_weight > max_weight/2:
          '''surgery: we delete the edge if its weight is more
          than a threshold, which is currently 0.5 percentile'''
          new_weight = 0

      new_data[x][y] = new_weight
      new_data[y][x] = new_weight
    old_data = new_data 
  return old_data

# trash

## Control

In [ ]:
control_csv = 'sub-OAS31172_ses-d0407_task-rest_run-2_atlas-BASC.csv'
patient_csv = 'sub-OAS31140_ses-d0332_task-rest_run-1_atlas-BASC.csv'

In [ ]:
control = data_from_csv(control_csv)
control_after_ricci = Ricci_Flow(control)

In [ ]:
L_control = laplacian(control_after_ricci, normed=True)

In [ ]:
w_control, v_control = LA.eig(L_control)

In [ ]:
import scipy.stats as st

class Gamma_control(st.rv_continuous):      
    def _pdf(self, x):
      g = 0
      global w_control
      sigma = 0.02
      for i in range(len(w_control)):
        g += np.exp(-(abs(x-w_control[i])**2)/(2*sigma**2)) / (np.sqrt(2*(np.pi)*(sigma**2)))
      return g/196.49999999999986

In [ ]:
gamma_control = Gamma_control(name='gamma_control')

In [ ]:
sample_control = gamma_control.rvs(size = 5)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: RuntimeWarning: overflow encountered in double_scalars
  if __name__ == '__main__':


RuntimeError: ignored

## Patient

In [ ]:
patient = data_from_csv(patient_csv)

In [ ]:
patient_after_ricci = Ricci_Flow(patient)

In [ ]:
L_patient = laplacian(patient_after_ricci, normed=True)

In [ ]:
w_patient, v_patient = LA.eig(L_patient)

In [ ]:
class Gamma_patient(st.rv_continuous):      
    def _pdf(self, x):
      g = 0
      global w_patient
      sigma = 0.02
      for i in range(len(w_patient)):
        g += np.exp(-(abs(x-w_patient[i])**2)/(2*sigma**2)) / (np.sqrt(2*(np.pi)*(sigma**2)))
      return g

In [ ]:
gamma_patient = Gamma_patient(name='gamma_patient')
sample_patient = gamma_patient.rvs(size = 5)

## EMD distance between patiet and control after RF

In [ ]:
print(wasserstein_distance(vector_control, vector_patient))

NameError: ignored

In [ ]:
import scipy.integrate as integrate
import scipy.special as special

def pdf_control(x):
      g = 0
      global w_control
      sigma = 0.02
      for i in range(len(w_control)):
        g += np.exp(-(abs(x-w_control[i])**2)/(2*sigma**2)) / (np.sqrt(2*(np.pi)*(sigma**2)))
      return g/196.49999999999986

result = integrate.quad(pdf_control, 0, np.inf)
result

(1.0000000000000002, 8.997889000979608e-09)

In [ ]:
lin = np.linspace(w.min(), w.max(), num=len(w))
plt.scatter(lin,w)

NameError: ignored

In [ ]:
s = set()
for i in old_data:
  for j in i:
    s.add(j)

In [ ]:
s = list(s)
lin = np.ones(len(s))

In [ ]:
import matplotlib.pyplot as plt

plt.scatter(s,lin)
plt.show()

In [ ]:
vector_control